In [9]:
import psycopg2
import sys
import pandas as pd
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import *
#from apscheduler.jobstores.sqlalchemy_store import SQLAlchemyJobStore

# follows django database settings format, replace with your own settings
DATABASES = {
'production':{
'NAME': 'xamplify',
'USER': 'postgres',
'PASSWORD': 'gjCj7fmExT6kq5j4',
'HOST': '138.68.44.49',
'PORT': 5432,
   },
}

# choose the database to use
db = DATABASES['production']

# construct an engine connection string
engine_string = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(
user = db['USER'],
password = db['PASSWORD'],
host = db['HOST'],
port = db['PORT'],
database = db['NAME'],
)

# create sqlalchemy engine
engine = create_engine(engine_string)

# read a table from database into pandas dataframe, replace "tablename" with your table name
df = pd.read_sql_table('xt_role',engine)

df

,role_id,role,description
0,1,ROLE_SUPER_ADMIN,None
1,2,ROLE_ORG_ADMIN,None
2,3,ROLE_USER,None
3,4,ROLE_VIDEO_UPLOAD,Uploading Video
4,5,ROLE_CAMPAIGN,Creating Campaign
5,6,ROLE_CONTACT,Uploading Contact
6,7,ROLE_EMAIL_TEMPLATE,Uploading Email Template
7,8,ROLE_STATS,Watching Statitics
8,9,ROLE_ALL,ALL Authorities
9,10,ROLE_SOCIAL_SHARE,Social Share Role


In [ ]:
import psycopg2
import sys
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

if len(sys.argv) < 1:
    print("Please provide database name and schema name")
    sys.exit(1)

print(sys.argv)

database = sys.argv[1]
host = sys.argv[2]
user = sys.argv[3]
password = sys.argv[4]

try:
    conn_source = psycopg2.connect(dbname="xamplify", host="138.68.44.49", user="postgres",
                                   password="gjCj7fmExT6kq5j4")
    conn_source.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    print('source connected successfully')
except Exception as e:
    print('cannot find the source')

cur_source = conn_source.cursor()
conn_source.commit()

try:
    conn_target = psycopg2.connect(dbname=database, host=host, user=user, password=password)
    conn_target.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    print('target connected successfully')
except Exception as e:
    print('cannot find the target')

cur_target = conn_target.cursor()
cur_target.execute("SELECT EXISTS (SELECT 1 AS result FROM pg_tables WHERE schemaname = 'xamplify_test_load' AND tablename = 'xt_role_d');")
tableExists = cur_target.fetchone()[0]
if tableExists==True: